In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [28]:
scraped = pd.read_pickle('data/scraped/scraped_with_decs.pickle.gz', compression='gzip')
scraped.head(3)

,name,image_path,price,url,raw_html,URL_name,description
0,Dom Perignon Lenny Kravitz Limited Edition wit...,/product/images/fl_progressive/lmgmud1xsenlouw...,199.97,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,Dom Pérignon Vintage 2008 has been distinguish...
1,Louis Roederer Cristal Brut with Two Flutes an...,/product/images/fl_progressive/iqec7e0nf6mzyof...,329.99,/product/Louis-Roederer-Cristal-Brut-with-Two-...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Louis-Roederer-Cristal-Brut-with-Two-...,Gift pack containing one bottle of 2008 Louis ...
2,Laurent-Perrier Cuvee Rose,/product/images/fl_progressive/10521.jpg,79.99,/product/Laurent-Perrier-Cuvee-Rose/10521,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Laurent-Perrier-Cuvee-Rose/10521,The Cuvée Rosé from Laurent-Perrier is the mos...


In [19]:
name_desc_txt = scraped[['name','description']]

In [22]:
name_desc_txt['start_token'] = "<|startoftext|>"
name_desc_txt['end_token'] = "<|endoftext|>"
name_desc_txt['prompt_token'] = "[prompt]"
name_desc_txt['response_token'] = "[response]"

name_desc_txt = name_desc_txt[
  [
    'start_token', 
    'prompt_token', 
    'name', 
    'response_token', 
    'description'
  ]
]

In [23]:
name_desc_txt.head()

,start_token,prompt_token,name,response_token,description
0,<|startoftext|>,[prompt],Dom Perignon Lenny Kravitz Limited Edition wit...,[response],Dom Pérignon Vintage 2008 has been distinguish...
1,<|startoftext|>,[prompt],Louis Roederer Cristal Brut with Two Flutes an...,[response],Gift pack containing one bottle of 2008 Louis ...
2,<|startoftext|>,[prompt],Laurent-Perrier Cuvee Rose,[response],The Cuvée Rosé from Laurent-Perrier is the mos...
3,<|startoftext|>,[prompt],Piper-Heidsieck Cuvee Brut in Travel Case with...,[response],#95 Wine Spectator Top 100 of 2019The Brut rep...
4,<|startoftext|>,[prompt],Clarendon Hills Astralis Syrah 2011,[response],"The pride of our portfolio, Clarendon Hills As..."


In [26]:
name_desc_txt

,start_token,prompt_token,name,response_token,description
0,<|startoftext|>,[prompt],Dom Perignon Lenny Kravitz Limited Edition wit...,[response],Dom Pérignon Vintage 2008 has been distinguish...
1,<|startoftext|>,[prompt],Louis Roederer Cristal Brut with Two Flutes an...,[response],Gift pack containing one bottle of 2008 Louis ...
2,<|startoftext|>,[prompt],Laurent-Perrier Cuvee Rose,[response],The Cuvée Rosé from Laurent-Perrier is the mos...
3,<|startoftext|>,[prompt],Piper-Heidsieck Cuvee Brut in Travel Case with...,[response],#95 Wine Spectator Top 100 of 2019The Brut rep...
4,<|startoftext|>,[prompt],Clarendon Hills Astralis Syrah 2011,[response],"The pride of our portfolio, Clarendon Hills As..."
...,...,...,...,...,...
125782,<|startoftext|>,[prompt],Whitehall Lane Tre Leoni 2014,[response],"This ferociously fun blend has depth, balance ..."
125783,<|startoftext|>,[prompt],Terrabianca Campaccio 2004,[response],"""The 2004 Campaccio (70% Sangiovese, 30% Caber..."
125784,<|startoftext|>,[prompt],Jean-Louis Chave Hermitage (1.5 Liter Magnum) ...,[response],
125785,<|startoftext|>,[prompt],Piper-Heidsieck Cuvee Brut (3.0 Liter Bottle -...,[response],#95 Wine Spectator Top 100 of 2019The Brut NV ...


In [24]:
train, test = train_test_split(name_desc_txt, test_size=0.2, random_state=0)
print(train.shape)
print(test.shape)

(100629, 5)
(25158, 5)


In [25]:
name_desc_txt.to_csv(
    "data/scraped/name_desc_nlp_ready.txt", 
    sep='\t',
    index=False,
    header=None)

train.to_csv(
    "data/scraped/name_desc_nlp_ready_train.txt", 
    sep='\t',
    index=False,
    header=None)

test.to_csv(
    "data/scraped/name_desc_nlp_ready_test.txt", 
    sep='\t',
    index=False,
    header=None)